<a href="https://colab.research.google.com/github/rafasacaan/dylan-gpt/blob/main/dylan_gpt_v2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install tiktoken

In [37]:
import os
import requests
import pandas as pd
import numpy as np
import tiktoken
import torch
import torch.nn as nn
from torch.nn import functional as F

In [38]:
# Check if GPU is available
if torch.cuda.is_available():
    device = torch.device("cuda")
    print("GPU is available.")
else:
    device = torch.device("cpu")
    print("GPU not available, using CPU.")

GPU is available.


In [39]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print(f"{device=}")

device=device(type='cuda')


In [40]:
# Set seed
torch.manual_seed(1337)

# Load Names data

Data found in kaggle [here](https://www.kaggle.com/datasets/cloudy17/bob-dylan-songs)

In [ ]:
# Load data
!wget https://raw.githubusercontent.com/karpathy/char-rnn/master/data/tinyshakespeare/input.txt

--2023-08-09 14:20:59--  https://raw.githubusercontent.com/karpathy/char-rnn/master/data/tinyshakespeare/input.txt
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.108.133, 185.199.109.133, 185.199.110.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.108.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 1115394 (1.1M) [text/plain]
Saving to: ‘input.txt.1’

input.txt.1         100%[===================>]   1.06M  --.-KB/s    in 0.02s   

2023-08-09 14:20:59 (57.6 MB/s) - ‘input.txt.1’ saved [1115394/1115394]



In [ ]:
with open('input.txt', 'r', encoding='utf-8') as f:
    text = f.read()

# Here are all the unique characters that occur in this text
chars = sorted(list(set(text)))
vocab_size = len(chars)

In [ ]:
print(vocab_size)

65


In [ ]:
print(chars)

['\n', ' ', '!', '$', '&', "'", ',', '-', '.', '3', ':', ';', '?', 'A', 'B', 'C', 'D', 'E', 'F', 'G', 'H', 'I', 'J', 'K', 'L', 'M', 'N', 'O', 'P', 'Q', 'R', 'S', 'T', 'U', 'V', 'W', 'X', 'Y', 'Z', 'a', 'b', 'c', 'd', 'e', 'f', 'g', 'h', 'i', 'j', 'k', 'l', 'm', 'n', 'o', 'p', 'q', 'r', 's', 't', 'u', 'v', 'w', 'x', 'y', 'z']


In [ ]:
# Create mapping from characters to integers and vice versa
stoi = { ch:i for i,ch in enumerate(chars) }
itos = { i:ch for i,ch in enumerate(chars) }
encode = lambda s: [stoi[c] for c in s]  # encoder: take a string, output a list of integers
decode = lambda l: ''.join([itos[i] for i in l]) # decoder: take a list of integers, output a string

In [ ]:
# Train and test splits
data = torch.tensor(encode(text), dtype=torch.long)
n = int(0.9 * len(data))
train_data = data[:n]
val_data = data[n:]

In [ ]:
train_data[:10]

tensor([18, 47, 56, 57, 58,  1, 15, 47, 58, 47])

# Hendrix lyrics

In [102]:
df = pd.read_csv('spotify_millsongdata.csv',  engine='python', error_bad_lines=False)

<ipython-input-102-265e00c0024e>:1: FutureWarning: The error_bad_lines argument has been deprecated and will be removed in a future version. Use on_bad_lines in the future.


  df = pd.read_csv('spotify_millsongdata.csv',  engine='python', error_bad_lines=False)
Skipping line 28662: unexpected end of data


In [103]:
df.shape

(28660, 4)

In [104]:
df

,artist,song,link,text
0,ABBA,Ahe's My Kind Of Girl,/a/abba/ahes+my+kind+of+girl_20598417.html,"Look at her face, it's a wonderful face \r\nA..."
1,ABBA,"Andante, Andante",/a/abba/andante+andante_20002708.html,"Take it easy with me, please \r\nTouch me gen..."
2,ABBA,As Good As New,/a/abba/as+good+as+new_20003033.html,I'll never know why I had to go \r\nWhy I had...
3,ABBA,Bang,/a/abba/bang_20598415.html,Making somebody happy is a question of give an...
4,ABBA,Bang-A-Boomerang,/a/abba/bang+a+boomerang_20002668.html,Making somebody happy is a question of give an...
...,...,...,...,...
28655,Dan Fogelberg,Our Last Farewell,/d/dan+fogelberg/our+last+farewell_20035714.html,Once I loved you and you loved me \r\nWe beli...
28656,Dan Fogelberg,Part Of The Plan,/d/dan+fogelberg/part+of+the+plan_20035777.html,I have these moments \r\nAll steady and stron...
28657,Dan Fogelberg,Promises Made,/d/dan+fogelberg/promises+made_20035748.html,Promises made \r\nPromises broken \r\nMeasur...
28658,Dan Fogelberg,Rhythm Of The Rain,/d/dan+fogelberg/rhythm+of+the+rain_20035806.html,[Chorus:] \r\nListen to the rhythm of the fal...


In [121]:
len(df.query("artist == 'Jimi Hendrix'").text.str.cat(sep='\n'))

26290

# Load Dylan Lyrics Data - Chars
Load data from [here](https://www.kaggle.com/datasets/cloudy17/bob-dylan-songs)

In [41]:
df = pd.read_csv('clear.csv')

In [42]:
# Check first lyric
text = df.lyrics[0]
text

'Come you ladies and you gentlemen, a-listen to my song\n\nSing it to you right, but you might think it’s wrong\n\nJust a little glimpse of a story I’ll tell\n\n’Bout an East Coast city that you all know well\n\nIt’s hard times in the city\n\nLivin’ down in New York town\n\n\n\nOld New York City is a friendly old town\n\nFrom Washington Heights to Harlem on down\n\nThere’s a-mighty many people all millin’ all around\n\nThey’ll kick you when you’re up and knock you when you’re down\n\nIt’s hard times in the city\n\nLivin’ down in New York town\n\n\n\nIt’s a mighty long ways from the Golden Gate\n\nTo Rockefeller Plaza ’n’ the Empire State.\n\nMister Rockefeller sets up as high as a bird\n\nOld Mister Empire never says a word\n\nIt’s hard times from the country\n\nLivin’ down in New York town\n\n\n\nWell, it’s up in the mornin’ tryin’ to find a job of work\n\nStand in one place till your feet begin to hurt\n\nIf you got a lot o’ money you can make yourself merry\n\nIf you only got a nick

In [43]:
# Aggregate lyrics into one string
text = (
  ''.join(
    df['lyrics']
    .str.replace('\n \n \n \n ', '\n\n\n')
    .str.replace('\n \n \n ', '\n\n\n')
    .str.replace('\n \n ', '\n\n')
    .str.replace('\n ', '\n')
    .str.replace('\n\n\n', '\n\n')
    .str.replace('\n\n', '\n')
  )
)

In [95]:
len(text)

523786

In [44]:
# Get char counts
char_count = {}

for char in text:
    if char in char_count:
        char_count[char] += 1
    else:
        char_count[char] = 1

In [45]:
sorted_char_count =  dict(sorted(char_count.items(), key=lambda item: item[1], reverse=True))

In [46]:
chars = sorted(list(set(text)))
vocab_size = len(chars)

In [47]:
vocab_size

91

In [48]:
# Create mapping from characters to integers and vice versa
stoi = { ch:i for i,ch in enumerate(chars) }
itos = { i:ch for i,ch in enumerate(chars) }
encode = lambda s: [stoi[c] for c in s]  # encoder: take a string, output a list of integers
decode = lambda l: ''.join([itos[i] for i in l]) # decoder: take a list of integers, output a string

In [75]:
# Train and test splits
#data = torch.tensor(encode(text), dtype=torch.long)
data = np.array(encode(text), dtype=np.int16)
n = int(0.9 * len(data))
train_data = data[:n]
val_data = data[n:]

In [76]:
train_data[:20]

array([26, 65, 63, 55,  2, 75, 65, 71,  2, 62, 51, 54, 59, 55, 69,  2, 51,
       64, 54,  2], dtype=int16)

In [77]:
decode(train_data[:100])

'Come you ladies and you gentlemen, a-listen to my song\nSing it to you right, but you might think it’'

In [78]:
print(''.join(stoi.keys()))


 !"'(),-.0123456789:;?ABCDEFGHIJKLMNOPQRSTUVWXYZ`abcdefghijklmnopqrstuvwxyz¥©éñóü–—‘’“”… 


# Load Dylan Lyrics Data - Words

In [6]:
df = pd.read_csv('clear.csv')

text = (
  ''.join(
    df['lyrics']
    .str.replace('\n \n \n \n ', '\n\n\n')
    .str.replace('\n \n \n ', '\n\n\n')
    .str.replace('\n \n ', '\n\n')
    .str.replace('\n ', '\n')
    .str.replace('\n\n\n', '\n\n')
    .str.replace('\n\n', '\n')
  )
)

In [7]:
import torchtext
from torchtext.data import get_tokenizer

# Tokenizer
tokenizer = get_tokenizer("basic_english")

In [8]:
text_tokenized = tokenizer(text)

In [37]:
tokens = sorted(list(set(text_tokenized)))
vocab_size = len(tokens)

vocab_size

8774

In [38]:
# Create mapping from characters to integers and vice versa
stoi = { ch:i for i,ch in enumerate(tokens) }
itos = { i:ch for i,ch in enumerate(tokens) }
encode = lambda s: [stoi[c] for c in s]  # encoder: take a string, output a list of integers
decode = lambda l: ''.join([itos[i] for i in l]) # decoder: take a list of integers, output a string

In [39]:
# Train and test splits
data = torch.tensor(encode(text_tokenized), dtype=torch.long)
n = int(0.9 * len(data))
train_data = data[:n]
val_data = data[n:]

In [40]:
train_data[:10]

tensor([1469, 8498, 4121,  298, 8498, 3089,    4,   70, 7620, 4910])

# Load Dylan Lyrics - Words (Karpathy)

In [6]:
df = pd.read_csv('clear.csv')

In [7]:
# Check first lyric
text = df.lyrics[0]
text

'Come you ladies and you gentlemen, a-listen to my song\n\nSing it to you right, but you might think it’s wrong\n\nJust a little glimpse of a story I’ll tell\n\n’Bout an East Coast city that you all know well\n\nIt’s hard times in the city\n\nLivin’ down in New York town\n\n\n\nOld New York City is a friendly old town\n\nFrom Washington Heights to Harlem on down\n\nThere’s a-mighty many people all millin’ all around\n\nThey’ll kick you when you’re up and knock you when you’re down\n\nIt’s hard times in the city\n\nLivin’ down in New York town\n\n\n\nIt’s a mighty long ways from the Golden Gate\n\nTo Rockefeller Plaza ’n’ the Empire State.\n\nMister Rockefeller sets up as high as a bird\n\nOld Mister Empire never says a word\n\nIt’s hard times from the country\n\nLivin’ down in New York town\n\n\n\nWell, it’s up in the mornin’ tryin’ to find a job of work\n\nStand in one place till your feet begin to hurt\n\nIf you got a lot o’ money you can make yourself merry\n\nIf you only got a nick

In [8]:
# Aggregate lyrics into one string
text = (
  ''.join(
    df['lyrics']
    .str.replace('\n \n \n \n ', '\n\n\n')
    .str.replace('\n \n \n ', '\n\n\n')
    .str.replace('\n \n ', '\n\n')
    .str.replace('\n ', '\n')
    .str.replace('\n\n\n', '\n\n')
    .str.replace('\n\n', '\n')
  )
)

In [9]:
# Split
n = len(text)
train_data = text[:int(n*0.9)]
val_data = text[int(n*0.9):]

n, len(train_data), len(val_data)

(523786, 471407, 52379)

In [10]:
# Encode with tiktoken gpt2 bpe
enc = tiktoken.get_encoding("gpt2")

# Encode
train_ids = enc.encode_ordinary(train_data)
val_ids = enc.encode_ordinary(val_data)

# Dtype
train_data = np.array(train_ids, dtype=np.uint16)
val_data = np.array(val_ids, dtype=np.uint16)

print(f"train has {len(train_data):,} tokens")
print(f"val has {len(val_data):,} tokens")

train has 136,916 tokens
val has 14,718 tokens


In [11]:
tokens = sorted(list(set(train_ids + val_ids)))
vocab_size = len(tokens)

vocab_size

8861

In [12]:
train_data[:10]

array([16773,   345, 17308,   290,   345, 28527,    11,   257,    12,
        4868], dtype=uint16)

In [13]:
# max token number
max(max(train_data),max(val_data))

50255

# Hyperparameters

In [26]:
# # Hyperparamters - gpu
# batch_size = 64      # number of independent sequences to process in parallel
# block_size = 256     # context length
# max_iters = 5_000    # training iterations
# eval_interval = 500  # how often to evaluate the loss on train and val sets
# eval_iters = 200     # how many iterations to average loss over when evaluating
# learning_rate = 3e-4 # learning rate param
# n_embed = 384         # embedding size
# n_layer = 6          # number of transformer blocks
# dropout = 0.2        # dropout rate
# n_head = 6           # number of heads
# device = 'cuda' if torch.cuda.is_available() else 'cpu'

In [18]:
# # Hyperparamters - cpu
# batch_size = 32      # number of independent sequences to process in parallel
# block_size = 16       # context length
# max_iters = 5_000    # training iterations
# eval_interval = 100  # how often to evaluate the loss on train and val sets
# eval_iters = 200     # how many iterations to average loss over when evaluating
# learning_rate = 1e-3 # learning rate param
# n_embed = 128         # embedding size
# n_layer = 4          # number of transformer blocks
# dropout = 0.0        # dropout rate
# n_head = 4           # number of heads
# device = 'cuda' if torch.cuda.is_available() else 'cpu'

RS

In [80]:
# Hyperparamters - gpu v1 - char format

out_dir = '/content/out-char'
eval_interval = 250  # how often to evaluate the loss on train and val sets
eval_iters = 200     # how many iterations to average loss over when evaluating
log_interval = 10    # dont print too often

# we expct to overfit: save when val improves
always_save_checkpoint = False

# logger
wandb_log = False
wandb_project = 'dylan-char'
wandb_run_name = 'mini-gpt'

gradient_accumulation_steps = 1
batch_size = 64      # number of independent sequences to process in parallel
block_size = 256     # context length


# baby gpt-model
learning_rate = 1e-4
max_iters = 5_000    # training iterations
lr_decay_iters = 5000
min_lr = 1e-4        # lr / 10
beta2 = 0.99         # make bigger because number of tokens per iters is small
warm_up_iters = 100

n_embed = 384        # embedding size
n_layer = 6          # number of transformer blocks
n_head = 6           # number of heads of each transformer block
dropout = 0.2        # dropout rate
device = 'cuda' if torch.cuda.is_available() else 'cpu'

# Batch data

In [81]:
def get_batch(split):
    """Generate a small random batch of data of inputs 'x'
    and targets 'y'
    """
    # get random index
    data = train_data if split == 'train' else val_data
    ix = torch.randint(
        low=0,
        high=len(data) - block_size,
        size=(batch_size,))

    # get the context for each index, and stack it into
    # rows.
    x = torch.stack([torch.from_numpy((data[i : i+block_size]).astype(np.int64)) for i in ix])
    y = torch.stack([torch.from_numpy((data[i+1 : i+1+block_size]).astype(np.int64)) for i in ix])


    #if device == 'cuda':
      # pin arrays x,y, which allows us to move them to GPU asynchronously (non_blocking=True)
      # x, y = x.pin_memory().to(device, non_blocking=True), y.pin_memory().to(device, non_blocking=True)

    #else:
    x, y = x.to(device), y.to(device)
    return x, y

In [82]:
x, y = get_batch(split='train')

In [83]:
len(x[0])

256

In [84]:
x[0], y[0]

(tensor([ 0, 32, 64,  2, 58, 55, 51, 72, 55, 64,  2, 65, 68,  2, 59, 64,  2, 58,
         55, 62, 62, 23,  0,  0, 24, 68, 55,  2, 75, 65, 71,  2, 68, 55, 51, 54,
         75,  8,  2, 51, 68, 55,  2, 75, 65, 71,  2, 68, 55, 51, 54, 75, 23,  0,
          0, 31, 51, 72, 55,  2, 75, 65, 71,  2, 57, 65, 70,  2, 69, 65, 63, 55,
          2, 71, 64, 56, 59, 64, 59, 69, 58, 55, 54,  2, 52, 71, 69, 59, 64, 55,
         69, 69, 23,  0, 32, 69,  2, 70, 58, 55, 68, 55,  2, 69, 65, 63, 55, 70,
         58, 59, 64, 57,  2, 58, 65, 62, 54, 59, 64, 57,  2, 75, 65, 71,  2, 52,
         51, 53, 61, 23,  0, 24, 68, 55,  2, 75, 65, 71,  2, 70, 58, 59, 64, 61,
         59, 64, 57,  2, 56, 65, 68,  2, 75, 65, 71, 68, 69, 55, 62, 56,  0, 38,
         68,  2, 51, 68, 55,  2, 75, 65, 71,  2, 56, 65, 62, 62, 65, 73, 59, 64,
         57,  2, 70, 58, 55,  2, 66, 51, 53, 61, 23,  0,  0, 24, 68, 55,  2, 75,
         65, 71,  2, 68, 55, 51, 54, 75,  8,  2, 58, 65, 66, 55,  2, 75, 65, 71,
         86, 68, 55,  2, 68,

# Estimate loss

In [85]:
@torch.no_grad()
def estimate_loss():
    """Averages out the loss over multiple batches, in order
    to get a better estimate of the loss.
    """
    out = {}

    # set model to evaluation phase
    model.eval()
    for split in ['train', 'val']:
        losses = torch.zeros(eval_iters)
        for k in range(eval_iters):
            X, Y = get_batch(split)
            logits, loss = model(X, Y)
            losses[k] = loss.item()
        out[split] = losses.mean()

    # set model to training phase
    model.train()
    return out

# Transformer `Head`

Define `Self Attention` model. The idea behind `self-attention` is to gather information from the past, in a data-dependant way.

For example,

$X_{4x8x32} * X_{32x16} = X_{4x8x16}$

where,

batch_size = 4

context_length = 8

embedding_dimension = 32

head_size = 16

In [86]:
class Head(nn.Module):
  """One head of self-attention.
  The idea is that each token has a:
    - key
    - query
    - value
  Then, for each token, the key queries every other token, and "resonates"
  with those where the link is stronger. These relationships are stored in
  a "weights" vector, which is applied over the value of each token, to weight
  each token in the past context.
  """

  def __init__(self, head_size):
    super().__init__()

    # Create independent and parallel key and query for each token
    self.key = nn.Linear(n_embed, head_size, bias=False)
    self.query = nn.Linear(n_embed, head_size, bias=False)
    self.value = nn.Linear(n_embed, head_size, bias=False)
    self.register_buffer(
        'tril',
        torch.tril(torch.ones(block_size, block_size))
    )
    self.dropout = nn.Dropout(dropout)


  def forward(self, x):
    B, T, C = x.shape
    k = self.key(x)   # (B, T, C) @ (C, head_size) -> (B, T, head_size)
    q = self.query(x) # (B, T, C) @ (C, head_size) -> (B, T, head_size)

    # Compute attention scores, i.e. affinities
    wei = q @ k.transpose(-1, -2) * C**-0.5 # (B, T, head_size) @ (B, head_size, T) -> (B, T, T)
    wei = wei.masked_fill(self.tril[:T, :T] == 0, float('-inf')) # (B, T, T)
    wei = F.softmax(wei, dim=-1) # (B, T, T)
    wei = self.dropout(wei)

    # Perform the weighted aggregation of the values
    v = self.value(x)
    out = wei @ v
    return out

In [87]:
class MulitHeadAttention(nn.Module):
  """Multiple heads of self-attention in parallel.
  Note: head_size * n_heads = n_embed
  """

  def __init__(self, num_heads, head_size):
    super().__init__()

    self.heads = nn.ModuleList(
        [Head(head_size) for _ in range(num_heads)]
    )

    self.proj = nn.Linear(n_embed, n_embed)
    self.dropout = nn.Dropout(dropout)


  def forward(self, x):
    # Concatenate the heads over the channel dimension, i.e. horizontally,
    # one besides the other, column-wise
    out = torch.cat(
        [h(x) for h in self.heads],
        dim=-1
    ) # (B, T, head_size * num_heads)
    out = self.dropout(self.proj(out))
    return out

In [88]:
class FeedForward(nn.Module):
  """A simple linear layer followed by a non-linearity
  """

  def __init__(self, n_embed):
    super().__init__()
    self.net = nn.Sequential(
        nn.Linear(n_embed, 4 * n_embed),
        nn.ReLU(),
        nn.Linear(4 * n_embed, n_embed),
        nn.Dropout(dropout)
    )

  def forward(self, x):
    return self.net(x)

In [89]:
class TransformerBlock(nn.Module):
  """Transformer block: communication followed by computation
  """

  def __init__(self, n_embed, n_head):
    super().__init__()
    head_size = n_embed // n_head
    self.sa = MulitHeadAttention(n_head, head_size)
    self.ffwd = FeedForward(n_embed)
    self.ln1 = nn.LayerNorm(n_embed)
    self.ln2 = nn.LayerNorm(n_embed)

  def forward(self, x):
    x = x + self.sa(self.ln1(x))
    x = x + self.ffwd(self.ln2(x))
    return x

# DylanGPT Model

In [90]:
class DylanGPT(nn.Module):

  def __init__(self, vocab_size):
    super().__init__()

    # Embeddings
    self.token_embedding_table = nn.Embedding(vocab_size, n_embed)
    self.postition_embedding_table = nn.Embedding(block_size, n_embed)

    # Transformer Blocks
    self.blocks = nn.Sequential(*[
        TransformerBlock(n_embed, n_head=n_head) for _ in range(n_layer)])

    # Final layer norm
    self.ln_f = nn.LayerNorm(n_embed)

    # LLM head
    self.lm_head = nn.Linear(n_embed, vocab_size)


  def forward(self, idx, targets=None):

    B, T = idx.shape
    token_embed = self.token_embedding_table(idx)
    pos_embed = self.postition_embedding_table(torch.arange(T, device=device))

    x = token_embed + pos_embed
    x = self.blocks(x)
    x = self.ln_f(x)
    logits = self.lm_head(x)

    if targets is None:
      loss = None
    else:
      # F.cross_entropy requires a (B, C, T) shape
      B, T, C = logits.shape
      logits = logits.view(B*T, C)
      targets = targets.view(B*T)
      loss = F.cross_entropy(logits, targets)

    return logits, loss


  def generate(self, idx, max_new_tokens):

    for _ in range(max_new_tokens):
      idx_cond = idx[:, -block_size:]

      logits, loss = self(idx_cond)

      logits = logits[:, -1, :]
      probs = F.softmax(logits, dim=-1)

      idx_next = torch.multinomial(probs, num_samples=1)
      idx = torch.cat((idx, idx_next), dim=1)
    return idx

# Train

In [91]:
%%time

# vocab size
#vocab_size = 50257

# model
model = DylanGPT(vocab_size=vocab_size)
m = model.to(device)

# optimizer
optimizer = torch.optim.AdamW(model.parameters(), lr=learning_rate)

# loop
for iter in range(max_iters):

    # every once in a while evaluate the loss on train and val sets
    if iter % eval_interval == 0:
        losses = estimate_loss()
        print(f"step {iter}: train loss {losses['train']:.4f}, val loss {losses['val']:.4f}")

    # sample a batch of data
    xb, yb = get_batch('train')

    # evaluate the loss
    logits, loss = model(xb, yb)
    optimizer.zero_grad(set_to_none=True)
    loss.backward()
    optimizer.step()

step 0: train loss 4.6613, val loss 4.6762
step 250: train loss 2.4087, val loss 2.4487
step 500: train loss 2.3588, val loss 2.4106
step 750: train loss 2.2637, val loss 2.3228
step 1000: train loss 2.0438, val loss 2.1289
step 1250: train loss 1.9045, val loss 2.0089
step 1500: train loss 1.7985, val loss 1.9187
step 1750: train loss 1.7158, val loss 1.8511
step 2000: train loss 1.6443, val loss 1.7969
step 2250: train loss 1.5958, val loss 1.7597
step 2500: train loss 1.5415, val loss 1.7264
step 2750: train loss 1.4981, val loss 1.6976
step 3000: train loss 1.4607, val loss 1.6858
step 3250: train loss 1.4202, val loss 1.6566
step 3500: train loss 1.3869, val loss 1.6411
step 3750: train loss 1.3545, val loss 1.6243
step 4000: train loss 1.3264, val loss 1.6216
step 4250: train loss 1.2963, val loss 1.6124
step 4500: train loss 1.2727, val loss 1.6092
step 4750: train loss 1.2410, val loss 1.5933
CPU times: user 18min 7s, sys: 14.8 s, total: 18min 22s
Wall time: 18min 17s


In [ ]:
# # Hyperparamters - gpu v1 - char format

# out_dir = '/content/out-char'
# eval_interval = 250  # how often to evaluate the loss on train and val sets
# eval_iters = 200     # how many iterations to average loss over when evaluating
# log_interval = 10    # dont print too often

# # we expct to overfit: save when val improves
# always_save_checkpoint = False

# # logger
# wandb_log = False
# wandb_project = 'dylan-char'
# wandb_run_name = 'mini-gpt'

# gradient_accumulation_steps = 1
# batch_size = 64      # number of independent sequences to process in parallel
# block_size = 256     # context length


# # baby gpt-model
# learning_rate = 1e-4
# max_iters = 5_000    # training iterations
# lr_decay_iters = 5000
# min_lr = 1e-4        # lr / 10
# beta2 = 0.99         # make bigger because number of tokens per iters is small
# warm_up_iters = 100

# n_embed = 384        # embedding size
# n_layer = 6          # number of transformer blocks
# n_head = 6           # number of heads of each transformer block
# dropout = 0.2        # dropout rate
# device = 'cuda' if torch.cuda.is_available() else 'cpu'

# step 0: train loss 4.6613, val loss 4.6762
# step 250: train loss 2.4087, val loss 2.4487
# step 500: train loss 2.3588, val loss 2.4106
# step 750: train loss 2.2637, val loss 2.3228
# step 1000: train loss 2.0438, val loss 2.1289
# step 1250: train loss 1.9045, val loss 2.0089
# step 1500: train loss 1.7985, val loss 1.9187
# step 1750: train loss 1.7158, val loss 1.8511
# step 2000: train loss 1.6443, val loss 1.7969
# step 2250: train loss 1.5958, val loss 1.7597
# step 2500: train loss 1.5415, val loss 1.7264
# step 2750: train loss 1.4981, val loss 1.6976
# step 3000: train loss 1.4607, val loss 1.6858
# step 3250: train loss 1.4202, val loss 1.6566
# step 3500: train loss 1.3869, val loss 1.6411
# step 3750: train loss 1.3545, val loss 1.6243
# step 4000: train loss 1.3264, val loss 1.6216
# step 4250: train loss 1.2963, val loss 1.6124
# step 4500: train loss 1.2727, val loss 1.6092
# step 4750: train loss 1.2410, val loss 1.5933
# CPU times: user 18min 7s, sys: 14.8 s, total: 18min 22s
# Wall time: 18min 17s

### To do:
- add plots for loss (train/val) and others
- find a good lr
- optimize initialization
- optimize learning rate

In [92]:
# Generate from model
context = torch.zeros((1,1),dtype=torch.long, device=device)
print(decode(m.generate(context, max_new_tokens=500)[0].tolist()))


When the devy them donce Hix
Just Can the triwn
But it’s a smile
We music out he down
Or go, ooh, let Ma, Mourca
La feet yel
I was the only lamen your manight poss and has about with they’re rulefuled
Love you wea! Evstis bad better hand to be new
Suble couldn’t leave you hear anyter
I know what it’s nuauth a eapose,one-deep

I creight pretty bus’ry on the churres
I can want Henry
Well, thenry I go back right to a trying flow
What to the car of the even certs at the bark
I coffee the boat of whe


Add temperature

In [93]:
T = 0.7 # (0,1])

idx = torch.zeros((1,1),dtype=torch.long, device=device)
for _ in range(500):
    idx_cond = idx[:, -block_size:]

    logits, loss = m(idx_cond)

    logits = logits[:, -1, :] * 1/T
    probs = F.softmax(logits, dim=-1)

    idx_next = torch.multinomial(probs, num_samples=1)
    idx = torch.cat((idx, idx_next), dim=1)

print(decode(idx[0].detach().cpu().numpy()))


Ill be a broken in the made of the streets
The pastander, standing has kinda in a proul of heart
And death of the steel of time
I was sing in a thine that wind
I would keep highold for from the wings like it to the wall
I was the gray fool, making miles home
Eatines I right for the eason to the slame
I was someone every with a tray that I knews the trun
I don’t eservery the storm fall
You know at sometimes of the strames away

Of a pattee sun if your home

Put your people that the fust it know
T


In [94]:
T = 0.6 # (0,1])

idx = torch.zeros((1,1),dtype=torch.long, device=device)
for _ in range(500):
    idx_cond = idx[:, -block_size:]

    logits, loss = m(idx_cond)

    logits = logits[:, -1, :] * 1/T
    probs = F.softmax(logits, dim=-1)

    idx_next = torch.multinomial(probs, num_samples=1)
    idx = torch.cat((idx, idx_next), dim=1)

print(decode(idx[0].detach().cpu().numpy()))


I’m a part of morals
An’ the world of my heart
In something the expeclains, he pulled of the was in here
But she ain’t right no to the Leven, the morning
I stand of the passion, the man sease the begins
She bearing the end song of a parkets

Well, the sing for at there’s was a sheem of a harm

I said, “Shadon you mad take it of the way know the was the parts
We lawer, the made of the man, my is hard a waiting white the back to see the story
Say me a door and I can see the stare of the fields of 
